In [1]:
import os
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize, LinearSegmentedColormap
from matplotlib import cm

# Set file paths
base_path = 'C:/Users/tasni/Results_Traffic_Rifa'
Area = 'WL'#options = 'Hyde', 'WL'


##############################################################
flows_path = os.path.join(base_path,Area, "Flows")
logs_path = os.path.join(base_path,Area, "Logs")
net_file_path = os.path.join(base_path,Area,"Nets_Trips")
visualization_path = os.path.join(base_path,Area, "Visualization")

data_paths = {
    "0.2m": os.path.join(base_path,Area,"Data","0.2m","link.csv"),
    "1m": os.path.join(base_path,Area,"Data","1m","link.csv"),
    "2m": os.path.join(base_path,Area,"Data","2m","link.csv"),
}

# Function to load flow data from text files
def load_flow_data(flow_file):
    flow_data = {}
    with open(flow_file, 'r') as f:
        lines = f.readlines()
        for i in range(0, len(lines), 2):  # Read every alternate line
            parts = lines[i].strip().split(' ')
            if len(parts) < 2:
                continue
            link, flow_value = parts[0], float(parts[1])
            from_id, to_id = map(int, link.strip('()').split(','))
            flow_data[(from_id, to_id)] = flow_value
    return flow_data

# Function to load capacity data from network files
def load_capacity_data(net_file):
    capacity_data = {}
    with open(net_file, 'r') as f:
        lines = f.readlines()
        metadata_end = False
        for line in lines:
            line = line.strip()
            if not line or line.startswith('~'):
                continue
            if '<END OF METADATA>' in line:
                metadata_end = True
                continue
            if metadata_end:
                parts = line.split()
                if len(parts) < 3 or not parts[0].isdigit():
                    continue
                init_node = int(parts[0])
                term_node = int(parts[1])
                capacity = float(parts[2])
                capacity_data[(init_node, term_node)] = capacity
    return capacity_data

# Process each dataset (0.2m, 1m, 2m)
for label, link_file in data_paths.items():
    if not os.path.exists(link_file):
        print(f"Skipping {label}, link.csv not found.")
        continue

    print(f"Processing {label} dataset...")

    # Load link data
    link_df = pd.read_csv(link_file)

    # Search for flow and net files in "Flows and logs"
    flow_files = [f for f in os.listdir(flows_path) if f.endswith("_flows.txt")]
    net_files = [f for f in os.listdir(net_file_path) if f.endswith("_net.txt")]
    
    for flow_file in flow_files:
        if any(flow_file.replace("_flows.txt", "") in net for net in net_files):
            net_file = next(net for net in net_files if flow_file.replace("_flows.txt", "") in net)

            # Load flow and capacity data
            flow_data = load_flow_data(os.path.join(flows_path, flow_file))
            capacity_data = load_capacity_data(os.path.join(net_file_path, net_file))

            # Map flow and capacity to the link DataFrame
            link_df['flow'] = link_df.apply(lambda row: flow_data.get((row['fromID'], row['toID']), 0), axis=1)
            link_df['capacity'] = link_df.apply(lambda row: capacity_data.get((row['fromID'], row['toID']), 0), axis=1)

            # Compute Flow-to-Capacity Ratio (FCR)
            link_df['FCR'] = link_df.apply(lambda row: row['flow'] / row['capacity'] if row['capacity'] > 0 else 0, axis=1)

            # Cap values at 5 and normalize between 0 and 5
            link_df['scaled_FCR'] = link_df['FCR'].apply(lambda x: min(x, 5))

            # Convert to GeoDataFrame
            gdf = gpd.GeoDataFrame(link_df, geometry=gpd.GeoSeries.from_wkt(link_df['geometry']))
            gdf.crs = 'EPSG:4326'

            # Define color scale (0-1.5: Green, 1.5-3: Yellow, 3-5: Red)
            colors = ['green', 'yellow', 'red']
            cmap = LinearSegmentedColormap.from_list("green_to_red", colors)
            norm = Normalize(vmin=0, vmax=5)

            # Plot
            fig, ax = plt.subplots(figsize=(12, 8))
            gdf.plot(
                ax=ax,
                color=gdf['scaled_FCR'].apply(lambda x: cmap(norm(x))),
                linewidth=2
            )

            # Add colorbar
            sm = cm.ScalarMappable(cmap=cmap, norm=norm)
            sm.set_array([])
            cbar = plt.colorbar(sm, ax=ax)
            cbar.set_label('Flow-to-Capacity Ratio (FCR)')

            # Titles and labels
            ax.set_title(f'Road Segments Colored by FCR ({label})', fontsize=16)
            ax.set_xlabel('Longitude', fontsize=12)
            ax.set_ylabel('Latitude', fontsize=12)

            # Save the figure
            output_file = os.path.join(visualization_path, flow_file+'.png')
            plt.savefig(output_file)
            plt.close()

            print(f"Visualization saved: {output_file}")

print("All visualizations complete.")


Processing 0.2m dataset...
Visualization saved: C:/Users/tasni/Results_Traffic_Rifa\WL\Visualization\WL_TR0.2_DR0.4_FF0.5_PF0.7_flows.txt.png
Visualization saved: C:/Users/tasni/Results_Traffic_Rifa\WL\Visualization\WL_TR0.2_DR0.4_FF0.6_PF0.8_flows.txt.png
Visualization saved: C:/Users/tasni/Results_Traffic_Rifa\WL\Visualization\WL_TR0.2_DR0.75_FF0.5_PF0.7_flows.txt.png
Visualization saved: C:/Users/tasni/Results_Traffic_Rifa\WL\Visualization\WL_TR0.2_DR0.75_FF0.6_PF0.8_flows.txt.png
Visualization saved: C:/Users/tasni/Results_Traffic_Rifa\WL\Visualization\WL_TR0.2_DR1_FF0.5_PF0.7_flows.txt.png
Visualization saved: C:/Users/tasni/Results_Traffic_Rifa\WL\Visualization\WL_TR0.2_DR1_FF0.6_PF0.8_flows.txt.png
Visualization saved: C:/Users/tasni/Results_Traffic_Rifa\WL\Visualization\WL_TR1_DR0.4_FF0.5_PF0.7_flows.txt.png
Visualization saved: C:/Users/tasni/Results_Traffic_Rifa\WL\Visualization\WL_TR1_DR0.4_FF0.6_PF0.8_flows.txt.png
Visualization saved: C:/Users/tasni/Results_Traffic_Rifa\WL